In [1]:
#Usar Python 2
from __future__ import division
import random
import math

import numpy as np
import matplotlib.pyplot as plt
from scipy.special import wofz as w
from scipy.optimize import curve_fit


#Se define la función de costo para utilizar en el proceso de optimización

def f_clx(x,f1,f2,f3,f4,f5,f6,
          G1,G2,G3,G4,G5,G6,
          wo1,wo2,wo3,wo4,wo5,wo6):   #wo: eV


    
    epsilon = 1

    epsilon += f1/((x**2-wo1**2)- (1j*G1*x)) #χ1
                   
    epsilon += f2/((x**2-wo2**2)- (1j*G2*x)) #χ1
                   
    epsilon += f3/((x**2-wo3**2)- (1j*G3*x)) #χ1
                   
    epsilon += f4/((x**2-wo4**2)- (1j*G4*x)) #χ1
                   
    epsilon += f5/((x**2-wo5**2)- (1j*G5*x)) #χ1
                   
    epsilon += f6/((x**2-wo6**2)- (1j*G6*x)) #χ1
    
        
    n=(np.sqrt(epsilon)).real
    k=(np.sqrt(epsilon)).imag
   
    return np.hstack([n, k])

def func2(x,f1,f2,f3,f4,f5,f6,
          G1,G2,G3,G4,G5,G6,
          wo1,wo2,wo3,wo4,wo5,wo6):  
   
    epsilon = 1

    epsilon += f1/((wo1**2-x**2)- (1j*G1*x)) #χ1
                   
    epsilon += f2/((wo2**2-x**2)- (1j*G2*x)) #χ1
                   
    epsilon += f3/((wo3**2-x**2)- (1j*G3*x)) #χ1
                   
    epsilon += f4/((wo4**2-x**2)- (1j*G4*x)) #χ1
                   
    epsilon += f5/((wo5**2-x**2)- (1j*G5*x)) #χ1
                   
    epsilon += f6/((wo6**2-x**2)- (1j*G6*x)) #χ1
    
   
    return epsilon

def opt_JC(xx):
    # Desempaquetar parámetros
    if len(xx) != 18:
        raise ValueError("La entrada xx debe tener 18 elementos, tiene {}".format(len(xx)))

    a1,a2,a3,a4,a5,a6 = xx[0:6]
    b1,b2,b3,b4,b5,b6 = xx[6:12]
    c1,c2,c3,c4,c5,c6 = xx[12:18]

    # Cargar datos experimentales
    try:
        file2 = np.loadtxt('AU_JC.txt')
    except Exception as e:
        print("Error al cargar datos:", e)
        return 1e10

    eV = file2[:,0]
    n = file2[:,1]
    k = file2[:,2]
    epsx = np.hstack([n, k])

    # Definir cotas para curve_fit
    ub = [a1,a2,a3,a4,a5,a6,b1,b2,b3,b4,b5,b6,c1,c2,c3,c4,c5,c6]
    lb = [0]*12 + [0.1]*6

    # Intentar curve_fit
    try:
        q, pcovBoth = curve_fit(
            f_clx, eV, epsx,
            bounds=(lb, ub),
            method='trf',
            tr_solver='lsmr',
            tr_options={'regularize': True},
            maxfev=10000
        )
    except Exception as e:
        print("curve_fit falló:", e)
        return 1e10  # valor alto para que PSO lo evite

    # Verificar longitud
    if len(q) != 18:
        print("curve_fit devolvió {} parámetros, se esperaba 18".format(len(q)))
        return 1e10

    # Calcular el error
    epsilon_model = func2(eV, *q)
    epst = ((epsilon_model.real - (n + 1j*k)**2).real)**2 + 50*((epsilon_model.imag - (n + 1j*k)**2).imag)**2
    testfunc = np.sum(epst)

    return np.abs(testfunc)

#Ciclo de optimización

#Se define la clase para crear las partículas del enjambre
class Particle:
    def __init__(self,x0):
        self.position_i=[]          # posición de la partícula
        self.velocity_i=[]          # velocidad de la partículaa
        self.pos_best_i=[]          # mejor posición individual
        self.err_best_i=-1         # mejor error individual
        self.err_i=-1              # error individual

        for i in range(0,num_dimensions):
            self.velocity_i.append(random.uniform(-1,1))
            self.position_i.append(x0[i])

    #Se evalúa la aptitud de la partícula usando la función de costo previamente definida
    def evaluate(self,costFunc):
        self.err_i=costFunc(self.position_i)

        #Revisa si la posición actual es la mejor y actualiza los valores de mejor posición y error 
        #individual
        if self.err_i < self.err_best_i or self.err_best_i==-1:
            self.pos_best_i=self.position_i
            self.err_best_i=self.err_i

    #Actualiza con la nueva velocidad de partícula que se generará con números aleatorios
    def update_velocity(self,pos_best_g):
        w=0.35       #Peso de inercia constante (cuánto influye la velocidad de la iteración anterior)
        c1=1.5        #Constante cognitiva (cuánto influye la mejor posición de la partícula)
        c2=2.5        #Constante social (cuánto influye la mejor posición del enjambre)

        for i in range(0,num_dimensions):
            r1=random.random()
            r2=random.random()

            vel_cognitive=c1*r1*(self.pos_best_i[i]-self.position_i[i])
            vel_social=c2*r2*(pos_best_g[i]-self.position_i[i])
            self.velocity_i[i]=w*self.velocity_i[i]+vel_cognitive+vel_social

    #Actualiza la posición de la partícula con la nueva velocidad
    def update_position(self,bounds):
        for i in range(0,num_dimensions):
            self.position_i[i]=self.position_i[i]+self.velocity_i[i]

            #Ajusta posición máxima si es necesario (si exceda cotas superiores)
            if self.position_i[i]>bounds[i][1]:
                self.position_i[i]=bounds[i][1]

            #Ajusta posición mínima si es necesario (si es menor a cotas inferiores)
            if self.position_i[i] < bounds[i][0]:
                self.position_i[i]=bounds[i][0]
                
class PSO():
    def __init__(self,costFunc,x0,bounds,num_particles,maxiter):
        global num_dimensions

        num_dimensions=len(x0)
        err_best_g=1000                   #Mejore error del enjambre
        pos_best_g=[]                    #Mejor posición del enjambre
        
        #Se crea el enjambre
        swarm=[]
        for i in range(0,num_particles):
            swarm.append(Particle(x0))

        #Inicia el ciclo de optimización
        i=0
        while i < maxiter:
            #Evaluar la aptitud de cada una de las partículas del enjambre
            for j in range(0,num_particles):
                swarm[j].evaluate(costFunc) 
                                #Se determina si la posición actual es la mejor del enjambre
                if swarm[j].err_i < err_best_g or err_best_g == 0:
                    pos_best_g=list(swarm[j].position_i)
                    err_best_g=float(swarm[j].err_i)
  
            #Actualizar las posiciones y velocidades del enjambre
            for j in range(0,num_particles):
                swarm[j].update_velocity(pos_best_g)
                swarm[j].update_position(bounds) 
            i+=1
            #Se muestra el mejor resultado hasta al momento (en cada iteración)
            print err_best_g
        #Se muestran los resultados finales
        print 'FINAL:'
        print err_best_g
        print("maxiter =", maxiter)

     

if __name__ == "__PSO__":
    main()

#Iniciar la optimización

#Parámetros para los valores iniciales del ajuste 
fs=20
Gs=3
wos=40
f1=0.1222
f2=0.2492
f3=2.7845
f4=0.1082
f5=7.8214
f6=10.5
wos1=3.57
wos2=4.21
wos3=9.15
wos4=2.92
wos5=38.96
wos6=15.5
G1=0.44
G2=0.97
G3=1.11
G4=0.46
G5=0.49
G6=0.53

#Valores para la posición inicial
initial=[f1,f2,f3,f4,f5,f6,
         G1,G2,G3,G4,G5,G6,
       wos1,wos2,wos3,wos4,wos5,wos6]              

#Cotas para el ciclo de optimización, deben estar contenidas en las cotas usadas para la función curve_fit
#bounds=[(0.00011,fs),(0.00011,fs),(0.00011,fs),(0.00011,fs),(0.00011,fs),(0.00011,fs),
#        (0.0011,Gs),(0.0011,Gs),(0.0011,Gs),(0.0011,Gs),(0.0011,Gs),(0.0011,Gs),
#        (2,wos),(3,wos),(8,wos),(2,wos),(35,wos),(12,wos)]

bounds=[(0.00011,fs),(0.00011,fs),(0.00011,fs),(0.00011,fs),(0.00011,fs),(0.00011,fs),
        (0.0011,Gs),(0.0011,Gs),(0.0011,Gs),(0.0011,Gs),(0.0011,Gs),(0.0011,Gs),
        (0.11,wos2),(0.11,wos),(0.11,wos2),(0.11,wos2),(0.11,wos4),(0.11,wos5)]


#Se escoge la cantidad de partículas del enjambre y las iteraciones
PSO(opt_JC,initial,bounds,num_particles=10,maxiter=1000)

#--- END ----------------------------------------------------------------------+

IndexError: list index out of range